# Machine Learning Analysis

This notebook applies machine learning algorithms to analyze the treatment starts dataset.

## Objectives
- Feature engineering
- Model selection and training
- Model evaluation
- Prediction and insights
- Comparison of different algorithms


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Load and preprocess data
df = pd.read_csv('../../data/mock_treatment_starts_2016.csv')
df['TreatmentStart'] = pd.to_datetime(df['TreatmentStart'], format='%m/%d/%y')
df['Year'] = df['TreatmentStart'].dt.year
df['Month'] = df['TreatmentStart'].dt.month
df['Day'] = df['TreatmentStart'].dt.day
df['Weekday'] = df['TreatmentStart'].dt.dayofweek
df['Quarter'] = df['TreatmentStart'].dt.quarter

print("Dataset loaded successfully!")
print(f"Shape: {df.shape}")
df.head()


## 1. Feature Engineering


In [ ]:
# Feature engineering
# Handle outlier (PT12 with dosage 1800 - likely data entry error, replace with median)
df_ml = df.copy()
outlier_idx = df_ml[df_ml['Dosage'] > 1000].index
if len(outlier_idx) > 0:
    df_ml.loc[outlier_idx, 'Dosage'] = df_ml['Dosage'].median()
    print(f"Outliers handled: {len(outlier_idx)} records")

# Encode categorical variables
le_drug = LabelEncoder()
df_ml['Drug_encoded'] = le_drug.fit_transform(df_ml['Drug'])

# Create features
X = df_ml[['Month', 'Day', 'Weekday', 'Quarter', 'Drug_encoded']].copy()
y = df_ml['Dosage'].copy()

print("=" * 60)
print("FEATURE ENGINEERING")
print("=" * 60)
print(f"\nFeatures: {list(X.columns)}")
print(f"Target: Dosage")
print(f"\nFeature shapes: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature statistics:")
print(X.describe())
print(f"\nTarget statistics:")
print(y.describe())


## 2. Data Splitting and Scaling


In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features (for algorithms that require scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("=" * 60)
print("DATA SPLITTING")
print("=" * 60)
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Train target mean: {y_train.mean():.2f}")
print(f"Test target mean: {y_test.mean():.2f}")


## 3. Model Training and Evaluation


In [ ]:
# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=1.0),
    'Decision Tree': DecisionTreeRegressor(random_state=42, max_depth=5),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, max_depth=5),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42, max_depth=3),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42, max_depth=3)
}

# Train and evaluate models
results = {}

print("=" * 60)
print("MODEL TRAINING AND EVALUATION")
print("=" * 60)

for name, model in models.items():
    # Use scaled data for linear models
    if name in ['Linear Regression', 'Ridge Regression', 'Lasso Regression']:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    }
    
    print(f"\n{name}:")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  MAE: {mae:.2f}")
    print(f"  R²: {r2:.4f}")

# Create results dataframe
results_df = pd.DataFrame(results).T
print("\n" + "=" * 60)
print("MODEL COMPARISON")
print("=" * 60)
print(results_df.round(4))


In [ ]:
# Visualize model performance
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. RMSE comparison
results_df_sorted = results_df.sort_values('RMSE')
axes[0, 0].barh(results_df_sorted.index, results_df_sorted['RMSE'], color='steelblue')
axes[0, 0].set_title('Model RMSE Comparison')
axes[0, 0].set_xlabel('RMSE')
axes[0, 0].grid(True, alpha=0.3, axis='x')

# 2. R² comparison
results_df_sorted_r2 = results_df.sort_values('R2', ascending=False)
axes[0, 1].barh(results_df_sorted_r2.index, results_df_sorted_r2['R2'], color='coral')
axes[0, 1].set_title('Model R² Comparison')
axes[0, 1].set_xlabel('R² Score')
axes[0, 1].grid(True, alpha=0.3, axis='x')

# 3. Best model predictions vs actual
best_model_name = results_df.idxmin()['RMSE']
if best_model_name in ['Linear Regression', 'Ridge Regression', 'Lasso Regression']:
    best_model = models[best_model_name]
    best_model.fit(X_train_scaled, y_train)
    y_pred_best = best_model.predict(X_test_scaled)
else:
    best_model = models[best_model_name]
    best_model.fit(X_train, y_train)
    y_pred_best = best_model.predict(X_test)

axes[1, 0].scatter(y_test, y_pred_best, alpha=0.6, color='teal')
axes[1, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1, 0].set_xlabel('Actual Dosage')
axes[1, 0].set_ylabel('Predicted Dosage')
axes[1, 0].set_title(f'Predictions vs Actual ({best_model_name})')
axes[1, 0].grid(True, alpha=0.3)

# 4. Residuals plot
residuals = y_test - y_pred_best
axes[1, 1].scatter(y_pred_best, residuals, alpha=0.6, color='purple')
axes[1, 1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1, 1].set_xlabel('Predicted Dosage')
axes[1, 1].set_ylabel('Residuals')
axes[1, 1].set_title(f'Residuals Plot ({best_model_name})')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nBest Model: {best_model_name}")
print(f"  RMSE: {results_df.loc[best_model_name, 'RMSE']:.2f}")
print(f"  R²: {results_df.loc[best_model_name, 'R2']:.4f}")


In [ ]:
# Feature importance (for tree-based models)
print("=" * 60)
print("FEATURE IMPORTANCE")
print("=" * 60)

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

tree_models = ['Decision Tree', 'Random Forest', 'Gradient Boosting', 'XGBoost']
for idx, model_name in enumerate(tree_models):
    if model_name in models:
        model = models[model_name]
        if model_name == 'XGBoost':
            model.fit(X_train, y_train)
        else:
            model.fit(X_train, y_train)
        
        if hasattr(model, 'feature_importances_'):
            importance = pd.DataFrame({
                'feature': X.columns,
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=False)
            
            row = idx // 2
            col = idx % 2
            axes[row, col].barh(importance['feature'], importance['importance'], color='steelblue')
            axes[row, col].set_title(f'Feature Importance: {model_name}')
            axes[row, col].set_xlabel('Importance')
            axes[row, col].grid(True, alpha=0.3, axis='x')
            
            print(f"\n{model_name}:")
            print(importance)

plt.tight_layout()
plt.show()


## 4. Cross-Validation


In [ ]:
# Cross-validation for best models
print("=" * 60)
print("CROSS-VALIDATION RESULTS")
print("=" * 60)

cv_results = {}
top_models = ['Random Forest', 'Gradient Boosting', 'XGBoost', 'Decision Tree']

for model_name in top_models:
    if model_name in models:
        model = models[model_name]
        cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
        cv_rmse = np.sqrt(-cv_scores)
        cv_results[model_name] = {
            'Mean RMSE': cv_rmse.mean(),
            'Std RMSE': cv_rmse.std(),
            'Scores': cv_rmse
        }
        print(f"\n{model_name}:")
        print(f"  Mean RMSE: {cv_rmse.mean():.2f} (+/- {cv_rmse.std() * 2:.2f})")

# Visualize CV results
cv_df = pd.DataFrame({k: v['Scores'] for k, v in cv_results.items()})
fig, ax = plt.subplots(figsize=(10, 6))
cv_df.boxplot(ax=ax)
ax.set_title('Cross-Validation RMSE Distribution')
ax.set_ylabel('RMSE')
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
